# Analyzing CIA World Factbook

### Aim

Use SQL in Jupyter Notebook to explore and analyze data from this database

### Dataset

A compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.
- name - The name of the country.
- area - The total land and sea area of the country.
- birth_rate - The country's birth rate, or the number of = births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
select * from sqlite_master
where type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Summary Statistics

In [3]:
%%sql
select * from facts limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [6]:
%%sql
select min(population),max(population),min(population_growth),max(population_growth) from facts;

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

In [9]:
%%sql
SELECT *
  FROM facts
 where population == (SELECT MIN(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [10]:
%%%sql 
select * from facts where population == (select max(population) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The country with no population is Antartica.

We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

In [11]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [13]:
%%sql
select avg(population), avg(area) from facts where name <> 'World';

Done.


avg(population),avg(area)
32242666.56846473,555093.546184739


In [18]:
%%sql
select * from facts 
where population > 
(select avg(population) from facts where name <> 'World')
and area <
(select avg(area) from facts where name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


In [20]:
%%sql
select * from facts where 
population  == (select max(population) from facts where name <> 'World')  

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


In [26]:
%%sql
select * from facts where 
population_growth  == (select max(population_growth) from facts where name <> 'World')  

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


In [35]:
%%sql
select * from facts where 
area_water/area_land  == (select max(area_water/area_land) from facts where name <> 'World')  

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None


In [36]:
%%sql
select * from facts where area_water>area_land;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


In [40]:
%%sql
select name, max(population*population_growth) from facts where name <> 'World'

Done.


name,max(population*population_growth)
India,1527068612.48


In [41]:
%%sql 
select name from facts where death_rate > birth_rate

Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


In [42]:
%%sql
select * from facts order by population/area desc limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0
